In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import random
from ortools.sat.python import cp_model
import time

In [2]:
#-----------------------------------EXTRACT THE DATA-----------------------------------
#start_time = time.time()

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
requests_path = '..\\eua-dataset\\users\\'
R = pd.read_csv(requests_path + 'users-test.csv') # test with 8 users
#R = pd.read_csv(requests_path + 'users-test-more-users.csv') # test with 14 users
#R = pd.read_csv(requests_path + 'users-melbcbd-generated.csv') # test with 816 users

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
nodes_path = '..\\eua-dataset\\edge-servers\\'
N = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
#N = pd.read_csv(nodes_path + 'serverstest-more-servers.csv') # test with 6 servers
#N = pd.read_csv(nodes_path + 'site-optus-melbCBD.csv') # test with 125 servers

In [3]:
#-----------------------------------INPUTS-----------------------------------
# Memory of function: m_f
m1 = 1
m2 = 2
m3 = 5
m4=8
m_f = [m1,m2,m3,m4]

#function assigned to each request
m_request = []
""" for j in range(len(R)):
  m = random.choice([m1, m2, m3,m4])
  m_request.append(m) """
m_request= [m1,m2,m3,m4,m1,m2,m3,m4]
#m_request= [m1,m2,m3,m4,m1,m2,m3,m4,m1,m2,m3,m4,m1,m2]
index = sorted(range(len(m_request)), key=lambda a: m_request[a]) # Sort the users by their requirements

#shows which function is assigned to each user
F=np.zeros([len(m_f),len(R)])
for m in range(len(m_f)):
  for j in range(len(R)):
    if m_request[j]==m_f[m]:
      F[m][j]=1
    else:
      F[m][j]=0  

#Maximum allowed network delay for function f:  phi_f
phi_f=[]


In [4]:
#-----------------------------------INFRASTRUCTURED DATA-----------------------------------

#Memory availably in node j: M_j
M_j = [8,9,10]

#Cores on node j: U_j
U_j=[45,46,47]

In [5]:
#-----------------------------------MONITORED DATA-----------------------------------
#Network delay between nodes i and j
delta_ij = []

#Incoming f requests to node i
lambda_fi = [[20,30,40,10,5,6,7,8],[20,30,40,10,5,6,7,8],[20,30,40,10,5,6,7,8]]

#Cores used by node j per single f request: u_fj
u_fj = [[5,6,7,8,1,2,3,4],[5,6,7,8,1,2,3,4],[5,6,7,8,1,2,3,4]]

In [6]:
#-----------------------------------VARIABLES-----------------------------------

U_si = [] # Set of individuals allocated to server s_i

#establish which containers (function instances) are allocated to each node
container_loc = [[1,0,1,0],[0,1,0,1],[1,0,0,1]]


In [7]:
#Groups requests by functions
G=[]
for row in range(len(F)):
    temp_G=[]
    for column in range(len(R)):
        if F[row][column]==1:
            temp_G.append(column)
    G.append(temp_G)


In [8]:
#-----------------------------------HAVERSINE-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [9]:
#-----------------------------------COVERAGE REQUEST-NODE-----------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N), 0.03)

for i in range(len(N)):
  server_latitude = N.iloc[i]['LATITUDE']
  server_longitude = N.iloc[i]['LONGITUDE']
  temp = []
  for j in range(len(R)):
    user_latitude = R.iloc[j]['Latitude']
    user_longitude = R.iloc[j]['Longitude']

    dist_geo = haversine(server_longitude, server_latitude, user_longitude, user_latitude)
    
    if dist_geo <= radius[i]:
        temp.append(1)
        
    else:
        temp.append(0)
       
  U_si.append(temp)


In [10]:
#---------------------------------DISTANCE BETWEEN NODES---------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N), 0.03)

for i in range(len(N)):
  server1_latitude = N.iloc[i]['LATITUDE']
  server1_longitude = N.iloc[i]['LONGITUDE']
  temp_dist = []
  for j in range(len(N)):
    server2_latitude = N.iloc[j]['LATITUDE']
    server2_longitude = N.iloc[j]['LONGITUDE']

    dist_geo_servers = haversine(server1_longitude, server1_latitude, server2_longitude,server2_latitude)
    #print(dist_geo)
    #print(radius[i]+radius[j])
    temp_dist.append(np.round(dist_geo_servers,3))
       
  delta_ij.append(temp_dist)

for r1 in range(len(N)):
  temp_delay = []
  for r2 in range(len(N)):
    if r1==r2:
      temp_delay.append(0)
    else:
      temp_delay.append(radius[r1]+radius[r2])
  phi_f.append(temp_delay)
    


In [11]:
#---------------------------------- CP MODEL ----------------------------------
model = cp_model.CpModel()

#-----------------------------------SOLVER VARIABLES-----------------------------------

# x_i,f = True if request f is allocated to node i
x = {}
for f in index:
    for i in range(len(N)):
        x[i, f] = model.NewBoolVar(f'c[{i}][{f}]')

# c_f,j = True if container of function f is deployed on node j
c = {}
for g in range(len(G)):
    for j in range(len(N)):
        c[g, j] = model.NewBoolVar(f'c[{g}][{j}]')

# y_i = True if node i is used
# y_i = False otherwise
y = {}
for i in range(len(N)):
    y[i] = model.NewBoolVar(f'c[{i}]')

print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]



In [12]:
prueba=[]

for ss1 in range(len(N)):
  temp_ss = []
  for ss2 in range(len(N)):
    if delta_ij[ss1][ss2]> phi_f[ss1][ss2]:
      temp_ss.append(0)
    else:
      temp_ss.append(1)
  prueba.append(temp_ss)

In [13]:
#-----------------------------------CONSTRAINTS-----------------------------------

# Proximity constraint (request-node)
for f in index:
    for i in range(len(N)):
        if U_si[i][f] == 0:
            model.Add(
             x[i, f]==0
            )

#Proximity constraint (node i-node j)
for i in range(len(N)):
    for j in range(len(N)):
        if delta_ij[i][j]> phi_f[i][j]:
            model.Add(
             x[i, f]==0
            )

#Container deployment: True if container of function f is deployed on node j
for f in index:
    for g in range(len(G)):
        for j in range(len(N)):
            model.Add(sum([x[i,f] for i in range(len(N))])<= c[g,j])

# Capacity constraint (memory)
for j in range(len(N)):
        model.Add(
            sum([
                m_f[g] * c[g,j] for g in range(len(G))
            ]) <= M_j[j]*y[j]
        )

# Avoid resource contention
for j in range(len(N)):
    model.Add(
            sum([
                sum([x[j,f]*lambda_fi[i][f]*u_fj[j][f] for f in index]) 
            for i in range(len(N))
            ]) <= U_j[j]
        )  


# Contraint family (each user can be allocated just once)
for f in index:
    model.Add(
        sum([
            x[i, f] for i in range(len(N))
        ]) <= 1
    ) 

print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 102 (#terms: 423)


In [17]:
#---------------------------------- CP SOLVER ----------------------------------
solver = cp_model.CpSolver()

#---------------------------------- OBJECTIVE FUNCTIONS ---------------------------------

# Maximize the number of allocated users
objective_max = []
for f in index:
    for j in range(len(N)):
        objective_max.append(x[j, f])
model.Maximize(sum(objective_max))

solver.Solve(model)
max_users = solver.ObjectiveValue()

# Hint (speed up solving)
for f in index:
    for j in range(len(N)):
        model.AddHint(x[j,f], solver.Value(x[j,f]))

for i in range(len(N)):
    model.AddHint(y[i], solver.Value(y[i]))
        
# Constraint previous objective
model.Add(
    sum([
        x[j, f] for i in range(len(N)) for j in index
    ]) == round(solver.ObjectiveValue())
)

# Minimize the number of servers used
objective_min = []
for i in range(len(N)):
    objective_min.append(y[i])
model.Minimize(sum(objective_min))

print(model.ModelStats())

IndexError: list index out of range

In [ ]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve(model)

#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    print(f'Objective value: {max_users} users have been allocated to {solver.ObjectiveValue()} servers\n')
    for f in index:
        for i in range(len(N)):
            if int(solver.Value(x[f,j])) == 1:
                print(f'User {j+1} has been allocated on server {i+1}')
else:
    print('The problem does not have an optimal solution.')

#print("\n--- Run time: %s seconds ---" % round((time.time() - start_time),2))

In [ ]:
"""
#-----------------------------------VARIABLES-----------------------------------
# All the requirements and capacities are set manually

U_si = [] #set of individuals allocated to server si
D = np.zeros((4), dtype=int) # D = [CPU, RAM, storage, bandwidth]

#User u has memory requirements = w (example for 8 users)
w1 = [1, 2, 1, 2]
w2 = [1, 2,	1, 2]
w3 = [5, 7,	6, 6]
w4 = [2, 3,	3, 4]
w5 = [5, 7,	6, 6]
w6 = [2, 3,	3, 4]
w7 = [1, 2, 1, 2]
w8 = [2, 3,	3, 4]
w9 = [1, 2, 1, 2]
w10 = [1, 2, 1, 2]
w11 = [5, 7, 6, 6]
w12 = [2, 3, 3, 4]
w13 = [5, 7, 6, 6]
w14 = [2, 3, 3, 4]
W = [w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14]
#W = [w1, w2, w3, w4, w5, w6, w7, w8]

# Each server has memory capacity c (example for 3 servers)
c1 = [12, 15, 15, 16]
c2 = [10, 17, 15, 16]
c3 = [10, 15, 11, 15]
c4 = [10, 16, 15, 17]
c5 = [12, 11, 12, 13]
c6 = [9, 13, 12, 14]
C = [c1, c2, c3, c4, c5, c6]
#C = [c1, c2, c3]

NORM = [LA.norm(w1), LA.norm(w2), LA.norm(w3), LA.norm(w4), LA.norm(w5), LA.norm(w6), LA.norm(w7), LA.norm(w8), LA.norm(w9), LA.norm(w10), LA.norm(w11), LA.norm(w12), LA.norm(w13), LA.norm(w14)]
#NORM = [LA.norm(w1), LA.norm(w2), LA.norm(w3), LA.norm(w4), LA.norm(w5), LA.norm(w6), LA.norm(w7), LA.norm(w8)]

index = sorted(range(len(NORM)), key=lambda a: NORM[a]) #increasing order of the index of user based on NORM
 """

In [ ]:
'''
#-----------------------------------VARIABLES-----------------------------------
# All the requirements and capacities are randomly generated

U_si = [] #set of individuals allocated to server si
D = np.zeros((4), dtype=int) # D = [CPU, RAM, storage, bandwidth]

# Users u has memory requirements = w_j
W = []
NORM = []
for j in range(len(U)):
  w = np.random.randint(1,7,4)
  W.append(w)
  norm = LA.norm(w)
  NORM.append(norm)

index = sorted(range(len(NORM)), key=lambda a: NORM[a])

# Each server has memory capacity C
C = []
for j in range(len(S)):
  c = np.random.randint(9,17,4)
  C.append(c)
'''